title: How big is the potential market for N26 Installments in France, Italy and Spain? (Part I)
author: Helder Silva 
date: 2021-07-16
region: EU
tags: N26 Installments, TBIL, bank products, credit, transactions, italy, france, spain, lisbon scores
summary:  In total, we found about 513K eligible users in these 3 markets. Even though Italy has the highest percentage of MAUs with eligible scores and transactions out of all MAUs in their population, France has by far the highest number of eligible MAUs (285.1K). Overall, users spent 957M€ in eligible transactions in the first half of 2021. Since France has the highest number of eligible users, it makes sense that they also have the highest transaction volume (583M€).

<div class="alert alert-block alert-success">
    <H1>How big is the potential market for N26 Installments in France, Italy and Spain? (Part I)</H1>
</div>

After the launch of N26 Instalments in Germany, we will be looking into how many potential users we have for this product in the French, Spanish, and Italian markets. In order to assess the potential eligibility of our users, we'll be looking into 2 main factors:
 - Users in these markets should have a favorable Lisbon score on June 30th 2021. Since credit scoring with Lisbon isn't yet live in these markets, we will be looking into Beta scores - for this we extracted CSV files with these scores directly from Lisbon since these aren't being passed into our DWH.
 - Also, they had at least 1 transaction between 50€ and 500€  in the first half of 2021 with a merchant that hasn't been excluded from this product. [Here](https://docs.google.com/spreadsheets/d/1Bn-0BHBPztNrjDj1yudYwDDjJFzLnmDXAbmEMApMFJ4/edit#gid=975436947) you can find a comprehensive list of the excluded MCCs.
 
We will be answering the following questions:
 - [How many users in these markets have a favorable score?](#section1)
 - [How many MAUs in these markets have eligible scores and transactions?](#section2)
 - [How much are eligible users spending in eligible transactions?](#section3)
 - [Which merchant categories get the biggest portion of the eligible transactions?](#section4)
 
The main findings of this research are:
 - In total, we found about 513K eligible users in these 3 markets. 
 - Even though Italy has the highest percentage of MAUs with eligible scores and transactions out of all MAUs in their population, France has by far the highest number of eligible MAUs (285.1K).
 - Overall, users spent 957M€ in eligible transactions in the first half of 2021. Since France has the highest number of eligible users, it makes sense that they also have the highest transaction volume (583M€). 
 - For all markets, users tend to have more transactions below 200€, which explains the fact that even though these buckets correspond to smaller volumes, they take the majority of the transaction volume. 

In [32]:
cd /app/

In [33]:
!pip install duckdb
!pip install altair

In [6]:
import pandas as pd
from utils.datalib_database import df_from_sql
import utils.altair_functions as af
import duckdb

con = duckdb.connect(database=":memory:", read_only=False)

In [7]:
esp_df = pd.read_csv(
    "research/product/bank_products/20210705_potential_european_market_TBIL/xgboost_user_pd_predictions_ESP.csv"
)
fra_df = pd.read_csv(
    "research/product/bank_products/20210705_potential_european_market_TBIL/xgboost_user_pd_predictions_FRA.csv"
)
ita_df = pd.read_csv(
    "research/product/bank_products/20210705_potential_european_market_TBIL/xgboost_user_pd_predictions_ITA.csv"
)

esp_df["market"] = "ESP"
fra_df["market"] = "FRA"
ita_df["market"] = "ITA"

input_df = pd.concat([esp_df, fra_df, ita_df])

<a id='section1'></a>
# How many users in these markets have a favorable score?

Below we can see that above 90% of users in each of the selected markets have a favorable Lisbon score. Even though Spain has the highest percentage of favorable scores, France has about 386.5K eligible users, more than Spain and Italy combined.

In [8]:
scores_query = """
select 
market,
round(count(case when rating_class <= 12 then 1 end)::numeric/1000, 1) as eligible_k_users,
round(count(case when rating_class > 12 then 1 end)::numeric/1000, 1) as non_eligible_k_users,
round(count(case when rating_class <= 12 then 1 end)::numeric/count(*)::numeric, 3)*100 as perc_eligible_users
from input_df 
group by 1
order by 1
"""
con.register("input_df", input_df)
scores_df = con.execute(scores_query).fetchdf()
scores_df

,market,eligible_k_users,non_eligible_k_users,perc_eligible_users
0,ESP,105.3,6.0,94.6
1,FRA,386.5,41.3,90.3
2,ITA,223.3,14.1,94.1


In [9]:
af.bar_single_label(
    scores_df, af.petrol, "perc_eligible_users", "market", 700, 200, "-x"
).properties(title="Ratio of Lisbon Eligible Users per country")

alt.LayerChart(...)

In [10]:
maus_txns_df = df_from_sql(
    "redshiftreader",
    """with maus_last_6_mo as (
select 
user_id, 
tnc_country_group
from dwh_cohort_weeks as d
inner join dbt.zrh_users as u
on d.end_time >= u.user_created
and d.end_time between '2021-01-01' and '2021-06-30'
and tnc_country_group in ('ESP', 'FRA', 'ITA')
inner join dbt.zrh_user_activity_txn as a
on a.user_created = u.user_created
and d.end_time between a.activity_start and least(u.closed_at,a.activity_end)
and activity_type = '1_tx_35'
group by 1, 2
),
eligible_txns_last_6_mo as ( 
select 
user_id,
count(*) as n_txns, 
sum(amount_cents_eur)::numeric/100 as sum_amount_eur,
sum_amount_eur::numeric/n_txns::numeric as avg_volume_eur_per_txn
from dbt.zrh_card_transactions
inner join dbt.zrh_users using(user_created)
where type = 'PT' 
and mcc not in (
'1343','1381','1454','1500','1761','1771','2490','3014','3028','3039',
'3048','3049','3267','3302','3351','3352','3353','3354','3356','3358',
'3359','3360','3361','3362','3363','3364','3365','3366','3367','3368',
'3369','3370','3371','3372','3373','3374','3375','3376','3377','3378',
'3379','3380','3381','3382','3383','3384','3385','3386','3387','3388',
'3390','3391','3392','3393','3394','3395','3396','3397','3398','3399',
'3400','3401','3402','3403','3404','3406','3407','3408','3409','3410',
'3411','3412','3413','3414','3415','3416','3417','3418','3419','3420',
'3421','3422','3423','3424','3425','3426','3427','3428','3429','3430',
'3431','3432','3433','3434','3435','3436','3437','3438','3439','3440',
'3520','3526','3544','3615','3635','3641','3670','3672','3716','3754',
'3777','3824','4121','4411','4511','4722','4812','4814','4816','4829',
'5039','5094','5122','5231','5310','5399','5411','5541','5812','5814',
'5815','5816','5912','5933','5960','5962','5963','5964','5965','5966',
'5967','5972','5983','5993','6011','6012','6051','6211','6513','6538',
'6540','7012','7021','7273','7277','7311','7361','7372','7399','7519',
'7841','7922','7988','7993','7994','7995','7997','8062','8071','8099',
'8211','8389','8999','9222','9223','9311','9399','9405','5813','5499',
'7523','8111','5948','5047','6300','5099','1520','7997','8299','3790') --Blacklisted MCCs provided by product
and created::date between '2021-01-01' and '2021-06-30'
and tnc_country_group in ('ESP', 'FRA', 'ITA')
and amount_cents_eur::numeric/100 between 50 and 499
group by 1
)
select *
from maus_last_6_mo 
left join eligible_txns_last_6_mo using (user_id)
""",
)

<a id='section2'></a>
# How many MAUs in these markets have eligible scores and transactions?

In total, we found about 513K eligible users in these 3 markets. Even though Italy has the highest percentage of MAUs with eligible scores and transactions out of all MAUs in their population, once again France has by far the highest number of eligible MAUs (285.1K).

In [11]:
users_query = """
with unions as (
select tnc_country_group as market, ' n_maus' as label, count(*)::numeric/1000 as n_users_k 
from maus_txns_df 
group by 1
union all 
select tnc_country_group, ' n_maus_elig_txns', round(count(case when n_txns is not null then 1 end)::numeric/1000, 1)
from maus_txns_df 
left join input_df i using (user_id)
group by 1
union all
select tnc_country_group, ' n_maus_elig_scores', round(count(case when i.user_id is not null then 1 end)::numeric/1000, 1)
from maus_txns_df 
left join input_df i using (user_id)
group by 1
union all
select tnc_country_group, 'n_maus_elig_scores_and_txns', round(count(case when i.user_id is not null and n_txns is not null then 1 end)::numeric/1000, 1)
from maus_txns_df 
left join input_df i using (user_id)
group by 1
), 
n_maus as (
select market, n_users_k as n_maus_k
from unions where trim(label) = 'n_maus'
)
select *,
round(n_users_k::numeric/ n_maus_k::numeric, 3)*100 as perc_users
from unions
inner join n_maus using(market)
"""
con.register("maus_txns_df", maus_txns_df)
grouped_users_df = con.execute(users_query).fetchdf()
grouped_users_df

,market,label,n_users_k,n_maus_k,perc_users
0,FRA,n_maus,650.416,650.416,100.0
1,ITA,n_maus,351.479,351.479,100.0
2,ESP,n_maus,185.242,185.242,100.0
3,FRA,n_maus_elig_txns,327.500,650.416,50.4
4,ITA,n_maus_elig_txns,178.300,351.479,50.7
5,ESP,n_maus_elig_txns,79.300,185.242,42.8
6,FRA,n_maus_elig_scores,423.900,650.416,65.2
7,ITA,n_maus_elig_scores,233.600,351.479,66.5
8,ESP,n_maus_elig_scores,110.000,185.242,59.4
9,FRA,n_maus_elig_scores_and_txns,285.100,650.416,43.8


In [34]:
af.column_multi(
    grouped_users_df, "market:N", "label:N", "perc_users:Q", "market:N", 100, 400, "x"
).properties(title="Ratio of eligible MAUs per country")

alt.Chart(...)

In [35]:
af.bar_single_label(
    grouped_users_df[grouped_users_df["label"] == "n_maus_elig_scores_and_txns"],
    af.petrol,
    "n_users_k:Q",
    "market",
    700,
    200,
    "-x",
).properties(title="Number of eligible MAUs per country")

alt.LayerChart(...)

In [14]:
elig_users_query = """ 
select user_id, 
trim(tnc_country_group) as tnc_country_group
from maus_txns_df
left join input_df i using (user_id)
where i.user_id is not null and n_txns is not null 
"""
elig_users_df = con.execute(elig_users_query).fetchdf()

In [15]:
fra_elig_users_ids = elig_users_df[
    elig_users_df["tnc_country_group"] == "FRA"
].user_id.tolist()
fra_elig_users_ids = ",".join(map("'{0}'".format, fra_elig_users_ids))

esp_ita_elig_users_ids = elig_users_df[
    elig_users_df["tnc_country_group"] != "FRA"
].user_id.tolist()
esp_ita_elig_users_ids = ",".join(map("'{0}'".format, esp_ita_elig_users_ids))

In [16]:
fra_mcc_df = df_from_sql(
    "redshiftreader",
    """
select 
tnc_country_group,
mcc_category,
case when amount_cents_eur::numeric/100 < 100 then '>= 50 and < 100'
when amount_cents_eur::numeric/100 < 200 then '>= 100 and < 200'
when amount_cents_eur::numeric/100 < 300 then '>= 200 and < 300'
when amount_cents_eur::numeric/100 < 400 then '>= 300 and < 400'
when amount_cents_eur::numeric/100 < 500 then '>= 400 and < 500'
end as loan_buckets,
count(*) as n_txns, 
count(distinct user_created) as n_users,
sum(amount_cents_eur)::numeric/100 as sum_amount_eur
from dbt.zrh_card_transactions 
inner join dbt.zrh_users using(user_created)
where amount_cents_eur::numeric/100 between 50 and 499
and mcc not in (
'1343','1381','1454','1500','1761','1771','2490','3014','3028','3039',
'3048','3049','3267','3302','3351','3352','3353','3354','3356','3358',
'3359','3360','3361','3362','3363','3364','3365','3366','3367','3368',
'3369','3370','3371','3372','3373','3374','3375','3376','3377','3378',
'3379','3380','3381','3382','3383','3384','3385','3386','3387','3388',
'3390','3391','3392','3393','3394','3395','3396','3397','3398','3399',
'3400','3401','3402','3403','3404','3406','3407','3408','3409','3410',
'3411','3412','3413','3414','3415','3416','3417','3418','3419','3420',
'3421','3422','3423','3424','3425','3426','3427','3428','3429','3430',
'3431','3432','3433','3434','3435','3436','3437','3438','3439','3440',
'3520','3526','3544','3615','3635','3641','3670','3672','3716','3754',
'3777','3824','4121','4411','4511','4722','4812','4814','4816','4829',
'5039','5094','5122','5231','5310','5399','5411','5541','5812','5814',
'5815','5816','5912','5933','5960','5962','5963','5964','5965','5966',
'5967','5972','5983','5993','6011','6012','6051','6211','6513','6538',
'6540','7012','7021','7273','7277','7311','7361','7372','7399','7519',
'7841','7922','7988','7993','7994','7995','7997','8062','8071','8099',
'8211','8389','8999','9222','9223','9311','9399','9405','5813','5499',
'7523','8111','5948','5047','6300','5099','1520','7997','8299','3790') --Blacklisted MCCs provided by product
and created::date between '2021-01-01' and '2021-06-30'
and user_id in("""
    + fra_elig_users_ids
    + """)
group by 1, 2, 3
""",
)

In [17]:
esp_ita_mcc_df = df_from_sql(
    "redshiftreader",
    """
select 
tnc_country_group,
mcc_category,
case when amount_cents_eur::numeric/100 < 100 then '>= 50 and < 100'
when amount_cents_eur::numeric/100 < 200 then '>= 100 and < 200'
when amount_cents_eur::numeric/100 < 300 then '>= 200 and < 300'
when amount_cents_eur::numeric/100 < 400 then '>= 300 and < 400'
when amount_cents_eur::numeric/100 < 500 then '>= 400 and < 500'
end as loan_buckets,
count(*) as n_txns, 
count(distinct user_created) as n_users,
sum(amount_cents_eur)::numeric/100 as sum_amount_eur
from dbt.zrh_card_transactions 
inner join dbt.zrh_users using(user_created)
where amount_cents_eur::numeric/100 between 50 and 499
and mcc not in (
'1343','1381','1454','1500','1761','1771','2490','3014','3028','3039',
'3048','3049','3267','3302','3351','3352','3353','3354','3356','3358',
'3359','3360','3361','3362','3363','3364','3365','3366','3367','3368',
'3369','3370','3371','3372','3373','3374','3375','3376','3377','3378',
'3379','3380','3381','3382','3383','3384','3385','3386','3387','3388',
'3390','3391','3392','3393','3394','3395','3396','3397','3398','3399',
'3400','3401','3402','3403','3404','3406','3407','3408','3409','3410',
'3411','3412','3413','3414','3415','3416','3417','3418','3419','3420',
'3421','3422','3423','3424','3425','3426','3427','3428','3429','3430',
'3431','3432','3433','3434','3435','3436','3437','3438','3439','3440',
'3520','3526','3544','3615','3635','3641','3670','3672','3716','3754',
'3777','3824','4121','4411','4511','4722','4812','4814','4816','4829',
'5039','5094','5122','5231','5310','5399','5411','5541','5812','5814',
'5815','5816','5912','5933','5960','5962','5963','5964','5965','5966',
'5967','5972','5983','5993','6011','6012','6051','6211','6513','6538',
'6540','7012','7021','7273','7277','7311','7361','7372','7399','7519',
'7841','7922','7988','7993','7994','7995','7997','8062','8071','8099',
'8211','8389','8999','9222','9223','9311','9399','9405','5813','5499',
'7523','8111','5948','5047','6300','5099','1520','7997','8299','3790') --Blacklisted MCCs provided by product
and created::date between '2021-01-01' and '2021-06-30'
and user_id in("""
    + esp_ita_elig_users_ids
    + """)
group by 1, 2, 3
""",
)

In [18]:
mcc_df = pd.concat([fra_mcc_df, esp_ita_mcc_df])
mcc_df["tnc_country_group"] = mcc_df["tnc_country_group"].str.strip()

<a id='section3'></a>
# How much are eligible users spending in eligible transactions?

Overall, users spent 957M€ in eligible transactions in the first half of 2021. Since France has the highest number of eligible users, it makes sense that they also have the highest transaction volume (583M€). For all markets, users tend to have more transactions below 200€, which explains the fact that even though these buckets correspond to smaller volumes, they take the majority of the transaction volume. 

In [19]:
txn_bucket_query = """
select
tnc_country_group as market, 
loan_buckets,
case when loan_buckets = '>= 50 and < 100' then ' >= 50 and < 100'
else loan_buckets end as loan_buckets,
round(sum(sum_amount_eur)::float/1000000::int) as total_txn_volume_M€, 
sum(sum_amount_eur)::numeric/ sum(n_users)::numeric as avg_volume_per_user, 
sum(n_txns)::numeric/ sum(n_users)::numeric as avg_txns_per_user, 
sum(sum_amount_eur)::numeric/ sum(n_txns)::numeric as avg_volume_per_txn
from mcc_df 
group by 1, 2, 3
order by 3
"""

In [20]:
con.register("mcc_df", mcc_df)
txn_bucket_df = con.execute(txn_bucket_query).fetchdf()
grouped_txns = txn_bucket_df.groupby(["market"]).sum().reset_index()
grouped_txns[["market", "total_txn_volume_m€"]]

,market,total_txn_volume_m€
0,ESP,103.0
1,FRA,583.0
2,ITA,271.0


In [36]:
af.column_multi(
    txn_bucket_df,
    "market:N",
    "loan_buckets:N",
    "total_txn_volume_m€:Q",
    "market:N",
    250,
    400,
    "x",
).properties(title="Volume of transactions per transaction bucket and country")

alt.Chart(...)

In [37]:
af.column_multi(
    txn_bucket_df,
    "market:N",
    "market:N",
    "avg_txns_per_user:Q",
    "loan_buckets:N",
    100,
    400,
    "x",
).properties(
    title="Average number of transactions per user per transaction bucket and country"
)

alt.Chart(...)

<a id='section4'></a>
# Which merchant categories get the biggest portion of the eligible transactions?

The answer to this question is quite different for each of the selected markets:
 - As for the total transaction volume, if we exclude the uncategorized merchant groups (no_cat), clothing department stores is the group with the highest volume for Spain and Italy (with 16.1M€ and 55.3M€ respectively), and the second highest in France (the highest merchant group in France is household stores with 109.1M€)

In [23]:
mcc_cat_query = """
select
tnc_country_group as market, 
case when trim(mcc_category) = 'clothing_depart_store' then 'clothing_dpt_store'
when trim(mcc_category) = 'computer_electronic_stores' then 'pc_electronic_stores'
when trim(mcc_category) = 'money_cash_financial' then 'money_cash_fin'
when trim(mcc_category) = 'local_transport_railway' then 'local_trans_railway'
when trim(mcc_category) = 'gambling_gaming' then 'gambling'
else trim(mcc_category) end as mcc_category,
round(sum(sum_amount_eur)::numeric/1000000, 1) as total_txn_volume_M€, 
sum(n_users),
round(sum(sum_amount_eur)::numeric/ sum(n_users)::numeric, 1) as avg_volume_per_user, 
round(sum(n_txns)::numeric/ sum(n_users)::numeric, 1) as avg_txns_per_user, 
round(sum(sum_amount_eur)::numeric/ sum(n_txns)::numeric) as avg_volume_per_txn
from mcc_df 
group by 1, 2
order by 2
"""

In [24]:
mcc_cat_df = con.execute(mcc_cat_query).fetchdf()

In [39]:
af.column_multi(
    mcc_cat_df,
    "market:N",
    "mcc_category:N",
    "total_txn_volume_m€:Q",
    "market:N",
    400,
    400,
    "-y",
).properties(
    title="Total transaction volume in million euros per merchant group and country"
)

alt.Chart(...)

In [26]:
mcc_cat_top10_query = """
with ita as (
select
tnc_country_group as market, 
mcc_category,
sum(sum_amount_eur) as total_txn_volume_M€, 
round(sum(sum_amount_eur)::numeric/ sum(n_users)::numeric, 1) as avg_volume_per_user, 
round(sum(n_txns)::numeric/ sum(n_users)::numeric, 1) as avg_txns_per_user, 
round(sum(sum_amount_eur)::numeric/ sum(n_txns)::numeric) as avg_volume_per_txn
from mcc_df 
where tnc_country_group = 'ITA'
group by 1, 2
order by 3 desc 
limit 10
), 
fra as(
select
tnc_country_group as market, 
mcc_category,
sum(sum_amount_eur) as total_txn_volume_M€, 
round(sum(sum_amount_eur)::numeric/ sum(n_users)::numeric, 1) as avg_volume_per_user, 
round(sum(n_txns)::numeric/ sum(n_users)::numeric, 1) as avg_txns_per_user, 
round(sum(sum_amount_eur)::numeric/ sum(n_txns)::numeric) as avg_volume_per_txn
from mcc_df 
where tnc_country_group = 'FRA'
group by 1, 2
order by 3 desc 
limit 10
), 
esp as (
select
tnc_country_group as market, 
mcc_category,
sum(sum_amount_eur) as total_txn_volume_M€, 
round(sum(sum_amount_eur)::numeric/ sum(n_users)::numeric, 1) as avg_volume_per_user, 
round(sum(n_txns)::numeric/ sum(n_users)::numeric, 1) as avg_txns_per_user, 
round(sum(sum_amount_eur)::numeric/ sum(n_txns)::numeric) as avg_volume_per_txn
from mcc_df 
where tnc_country_group = 'ESP'
group by 1, 2
order by 3 desc 
limit 10
)
select * from ita 
union all select * from fra
union all select * from esp
"""

In [27]:
mcc_cat_top10_df = con.execute(mcc_cat_top10_query).fetchdf()

 After identifying the top 10 merchant groups in each country based on the volume of transactions, we will look into these on the user/ transaction level:
 - As for the average number of transactions per user for the top 10 groups, the gas service station group is the highest one in France with an average of 4.6 transactions, as well as for Italy (5.5 transactions on average), and car clothing department store leads in Spain (with 3.3 transactions per user).
 - In regards to the average volume per transaction, business org. services is on top for France (with 147€ per transaction), utilities leads in Italy (also with 147€ on average), and car rental has the highest average in Spain (162€).

In [38]:
af.column_multi(
    mcc_cat_top10_df,
    "market:N",
    "mcc_category:N",
    "avg_txns_per_user:Q",
    "market:N",
    250,
    400,
    "x",
).properties(
    title="Avg Transactions per user for the top 10 merchant groups per country"
)

alt.Chart(...)

In [29]:
af.column_multi(
    mcc_cat_top10_df,
    "market:N",
    "mcc_category:N",
    "avg_volume_per_txn:Q",
    "market:N",
    250,
    400,
    "x",
).properties(title="Avg Volume per txn for the top 10 merchant groups per country")

alt.Chart(...)

# Appendix

### Could we use Klarna transactions as a proxy for the appetite for this product in the selected markets?

By looking at direct debit requests with the keyword 'klarna' in the creditor name, we found a very small sample of users using this product outside of Germany in the first half of 2021. 

In [30]:
klarna_txns_df = df_from_sql(
    "redshiftreader",
    """
select
tnc_country_group,
count(*) as n_tnxs,
count(distinct user_id) as n_users
from etl_reporting.ch_direct_debit_request_incoming dd
inner join etl_reporting.cr_account_external_id ea
on dd.debitor_iban = ea.external_id
inner join dbt.zrh_users using (account_id)
where creditor_name ilike '%KLARNA%'
and (timestamp 'epoch' + dd.created/1000 * INTERVAL '1 Second ') between '2021-01-01' and '2021-06-30'
group by 1
order by 2 desc
""",
)

In [31]:
klarna_txns_df

,tnc_country_group,n_tnxs,n_users
0,DEU,282244,33084
1,AUT,7072,1833
2,GrE,1976,404
3,NEuro,690,107
4,ITA,576,105
5,FRA,568,82
6,ESP,389,74
